In [ ]:
%pip install -qU langchain langchain-openai

%pip install langchain-google-genai

In [88]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_core.prompts import ChatPromptTemplate

os.environ['GOOGLE_API_KEY'] = "api_key"

# Initialize an LLM
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-latest",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,)

# see the list of models here : https://ai.google.dev/gemini-api/docs/models/gemini


In [25]:
import datetime
from typing import Literal, Optional, Tuple

from langchain_core.pydantic_v1 import BaseModel, Field


class SubQuery(BaseModel):
    """Subtask of the original query which can't be broken further, and that which can be solved by an expert finetuned LLM."""

    sub_query: str = Field(
        ...,
        description="The minimal and specific subtask that can be solved by an expert finetuned LLM.",
    )

In [84]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI

system = """You are a helpful assistant that prepares queries that will be sent to a search component.
Sometimes, these queries are very complex.
Your job is to simplify complex queries into multiple queries that can be answered
in isolation to eachother.

If the query is simple, then keep it as it is.
Examples
1. Query: Did Microsoft or Google make more money last year?
   Decomposed Questions: [Question(question='How much profit did Microsoft make last year?', answer=None), Question(question='How much profit did Google make last year?', answer=None)]
2. Query: What is the capital of France?
   Decomposed Questions: [Question(question='What is the capital of France?', answer=None)]
3. Query: {{question}}
   Decomposed Questions:
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

llm_with_tools = llm.bind_tools([SubQuery])
parser = PydanticToolsParser(tools=[SubQuery])
query_analyzer = prompt | llm_with_tools | parser

In [85]:
question = 'What is the capital of France? Is it the birthplace of Napoleon? Was he ever the emperor?'
result = query_analyzer.invoke(
    {
        "question": question
    }
)
result

[SubQuery(sub_query='What is the capital of France?'),
 SubQuery(sub_query='Is the capital of France the birthplace of Napoleon?'),
 SubQuery(sub_query='Was Napoleon ever the emperor of France?')]

In [87]:
question = "Who is the current US President's wife?"
result = query_analyzer.invoke(
    {
        "question": question
    }
)
result

[SubQuery(sub_query="Who is the current US President\\'s wife?")]